# Setup

In [1]:
backend_name = 'ibm_kawasaki'

In [2]:
import icepp
import csv
import math
from decimal import Decimal, ROUND_HALF_UP, ROUND_HALF_EVEN

from sample.onestepSim_LBNL import runQuantum as step_1
from sample.twostepSim_LBNL import runQuantum as step_2
from pytket.extensions.qiskit import qiskit_to_tk, tk_to_qiskit
from pytket.extensions.qiskit import IBMQBackend, AerStateBackend, AerBackend

from qiskit import *
IBMQ.load_account()
from pytket.extensions.qiskit import IBMQBackend, AerStateBackend, AerBackend

# Fill your provider information
# Example: provider = IBMQ.get_provider(hub=ibm-q'',group='open',project='main')
provider = IBMQ.get_provider(hub='',group='',project='')

backend = provider.get_backend(backend_name)
sim_b = AerBackend()
dev_b = IBMQBackend(backend_name=backend_name, hub='',group='',project='')

In [3]:
def decompose_to_basis(qc):
    
    qc = transpile(qc, basis_gates=['id','x','sx','rz','cx','reset'])
    
    return qc

def decompose_to_toffoli(qc):

    qc = icepp.decompose().decompose_mcu(qc)
    
    return qc


def tket_opt(qc, backend):
    
    tket_qc = qiskit_to_tk(qc)
    
    if backend == None:
        backend = sim_b
    else:
        backend = dev_b
        
    backend.compile_circuit(tket_qc, optimisation_level=2)
    qc = tk_to_qiskit(tket_qc)
    
    return qc


def qiskit_opt(qc, backend):
    
    if backend == None:
        qc = transpile(qc, basis_gates=['id','x','sx','rz','cx','reset'], optimization_level=3)
    else:
        qc = transpile(qc, backend, seed_transpiler=1, basis_gates=['id','x','sx','rz','cx','reset'], optimization_level=3)
    
    return qc


def aqcel_opt(qc):
    
    qc = icepp.pass_manager(qc, measure_type='cc').aqcel_apply()
    
    return qc


def manager(qc, backend, method_list):
    
    for method in method_list:
        
        if method == 't':
            qc = tket_opt(qc, backend)
        if method == 'u':
            qc = tket_opt(qc, None)
            
        if method == 'q':
            qc = qiskit_opt(qc, backend)
        if method == 'r':
            qc = qiskit_opt(qc, None)
            
        if method == 'd':
            qc = decompose_to_basis(qc)
        if method == 'e':
            qc = decompose_to_toffoli(qc)
            
        if method == 'a':
            qc = aqcel_opt(qc)
            
    return qc


def apply(qc, backend, method_list):
    
    method_dic = {}
    for method in method_list:
        new_qc = manager(qc, backend, method)
        method_dic[method] = new_qc
        
    return method_dic

def apply_optimizer(ps, backend, sim_list, qc_list):
 
    sim_qcs = apply(ps, None,    sim_list)
    qc_qcs  = apply(ps, backend, qc_list)
    
    """
    print(analysis(ps))
    for method in sim_list:
        qc = sim_qcs[method]
        print(analysis(qc))
    for method in qc_list:
        qc = qc_qcs[method]
        print(analysis(qc))
    """
    
    return sim_qcs, qc_qcs

In [4]:
def run(step, backend, sim_list, qc_list, shots):
    
    if step == 1:
        ps = step_1(gLR=1,dophisplit=1)
    if step == 2:
        ps = step_2(gLR=1,dophisplit=1)
    
    sim_qcs, qc_qcs = apply_optimizer(ps, backend, sim_list, qc_list)
    
    qc_result_dic = experiment(step, qc_qcs, backend, shots)
    simulation = dic_size(ps)
        
    final_result = []

    for key in sim_list:
        qc = sim_qcs[key]
        counts = transpile(qc, basis_gates=['id','x','sx','rz','cx','reset'], optimization_level=0).count_ops()
        Fsim = fidelity(simulation, dic_size(qc) ,1)
        key_result = [key, qc.data, None, counts, None, qc.num_qubits, None, Fsim, None]
        final_result.append(key_result)

    
    for key in qc_list:
        qc = qc_qcs[key]
        counts = analysis(qc)
        results = qc_result_dic[key]
        
        fidelities=[]
        for x in range(30):
            raw = results.get_counts(x)
            fidelities.append( fidelity(simulation,raw,shots) )
            
        key_result = [key, None, qc.data, counts[1], counts[0], None, results, None, fidelities]
        
        final_result.append(key_result)
        

    csv_generator(step, final_result)

    return [sim_qcs, qc_qcs, qc_result_dic, final_result]


def experiment(step, qc_qcs, backend, shots):
    
    qc_result_dic = {}
    for key in qc_qcs.keys():
        
        results = icepp.qc_experiment(qc_list=[qc_qcs[key]]*30, backend=backend, shots=shots).run()
        
        qc_result_dic[key]=results
        
    return qc_result_dic

def csv_generator(step, final_result):
    header = ['name', 'original circuit', 'transpiled circuit', 'gate counts', 'depth', 'qubits', 'raw counts', 'F_sim', 'F_meas']
    # Job_is future work
    body = final_result

    if step == 1:
        step_str = '_1step_cc.csv'
    if step == 2:
        step_str = '_2step_cc.csv'
    
    #"""
    with open(backend_name+step_str, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(body)
    """
    with open(backend_name+step_str, 'a') as f:
        writer = csv.writer(f)
        writer.writerows(body)
    """

    f.close()

def dic_size(qc):
    
    sim_result = icepp.simulator(qc).statevector_simulator(output='probabilities_distribution')
    
    new_sim_result = key_changer(qc, sim_result)
        
    return new_sim_result

def key_changer(qc, dic):
    measure_list = [0]*qc.num_clbits
    for op in qc.get_instructions('measure'):
        measure_list[op[2][0]._index] = qc.qubits.index(op[1][0])
    
    new_dic = {}
    for key in dic.keys():
        new_key = ''
        for index in reversed(measure_list):
            new_key = new_key + key[qc.num_qubits - index -1]
            
        if new_key in new_dic.keys():
            new_dic[new_key] = new_dic[new_key] + dic[key]
        else:
            new_dic[new_key] = dic[key]
        
    return new_dic

def fidelity(count1,count2,shots):
    fidelity=0
    for key in count1.keys():
        if (key in count2) and (count1[key] >0) and (count2[key] >0):
            fidelity += math.sqrt(count1[key]*count2[key]/shots)
    return float(Decimal(str(fidelity)).quantize(Decimal('0.00001'), rounding=ROUND_HALF_UP))

def analysis(qc):
    
    tr_qc = transpile(qc,basis_gates=['id','x','sx','rz','cx','reset'], optimization_level=0)
    time = schedule(qc, backend).duration*backend.configuration().dt
    
    return [time, tr_qc.count_ops()]

# Optimize benchmark by AQCEL, t|ket> and Qiskit

In [5]:
sim_list = ['e', 'edu', 'er', 'eru', 'a', 'aru']
qc_list  = ['eq', 'edt', 'edtq', 'adtq']
"""
sim_list = ['e', 'edu', 'er', 'eru', 'a', 'adu', 'ar', 'aru']
qc_list  = ['eq', 'edt', 'edtq', 'aq', 'adt', 'adtq']
"""

"\nsim_list = ['e', 'edu', 'er', 'eru', 'a', 'adu', 'ar', 'aru']\nqc_list  = ['eq', 'edt', 'edtq', 'aq', 'adt', 'adtq']\n"

In [6]:
package1 = run(step=1, backend=backend, sim_list=sim_list, qc_list=qc_list, shots=10000)

Job id : c7deed884ed14927a802fa3ddfb1306d-1648615783311928
Job Status: job has successfully run
Job id : 6e2c2a0eac87450ebc5f080565677808-16486161218252969
Job Status: job has successfully run
Job id : 4ac0304d82ed4efc8cfbae292b2d7f50-16486163915170848
Job Status: job has successfully run
Job id : 2859941669694439add20d38ce59db2c-1648616630935053
Job Status: job has successfully run
